In [1]:
import os
os.chdir("..")

In [2]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [3]:
from mlengine.data_read.read import read_csv_file
from mlengine.config.settings import settings

2023-11-14 23:01:04 DESKTOP-PEC6JQ0 11_14_2023_23_01_04.log[24848] INFO Logging initialized.
******************************
2023-11-14 23:01:04 DESKTOP-PEC6JQ0 11_14_2023_23_01_04.log[24848] INFO Settings file [d:\PycharmProjects\ml-app\src\mlengine\config\settings.yaml] loaded successfully.
******************************


In [4]:
df = read_csv_file(os.path.join(settings.data_ingestion.root_dir, settings.data_ingestion.data_file))
df

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [5]:
from sklearn.model_selection import train_test_split
target = 'math_score'

In [6]:
def get_X_y(df, target):
    y = df[target]
    X = df.drop(target, axis=1)
    return X, y

In [7]:
def get_train_validate_test_X_y(X, y, test_size=0.4, *args, **kwargs):
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=None, test_size=test_size, *args, **kwargs)
    X_validate, X_test, y_validate, y_test = train_test_split(X_test, y_test, stratify=None, test_size=0.5, *args, **kwargs)
    return X_train, X_validate, X_test, y_train, y_validate, y_test

In [18]:
X, y = get_X_y(df, target)

In [19]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [20]:
preprocessor

ColumnTransformer(transformers=[('OneHotEncoder', OneHotEncoder(),
                                 Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course'],
      dtype='object')),
                                ('StandardScaler', StandardScaler(),
                                 Index(['reading_score', 'writing_score'], dtype='object'))])

In [21]:
X_train, X_validate, X_test, y_train, y_validate, y_test = get_train_validate_test_X_y(X, y)

In [22]:
rfe = RFE(SVR(kernel="linear"), step=1)
prep_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

In [23]:
X_train = prep_pipeline.fit_transform(X_train, y_train)
X_validate = prep_pipeline.transform(X_validate)
X_test = prep_pipeline.transform(X_test)

In [24]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [25]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

In [26]:
model_pipelines = dict()

for name, clf in models.items():
    # model_pipelines[name] = Pipeline(steps=[('rfe', rfe), (name, clf)])
    model_pipelines[name] = Pipeline(steps=[(name, clf)])

In [27]:
def foo(y, y_pred):
    mae, rmse, r2 = evaluate_model(y, y_pred)
    print("- Root Mean Squared Error: {:.4f}".format(rmse))
    print("- Mean Absolute Error: {:.4f}".format(mae))
    print("- R2 Score: {:.4f}".format(r2))

In [28]:
model_list = []
# r2_list =[]

for name, model_pipeline in model_pipelines.items():
    # model_pipeline.fit(X_train, y_train) # Train model
    models = GridSearchCV(model_pipeline, param_grid={}, cv=5, verbose=True)
    best_model = models.fit(X_train, y_train).best_estimator_

    best_model.fit(X_train, y_train)  # retrain the model again on full training data (previously we were 1 fold short for each iter of CV)

    model_list.append(best_model)

    # Make predictions
    y_train_pred = best_model.predict(X_train)
    y_validate_pred = best_model.predict(X_validate)

    print(best_model)

    print('Model performance for Training set')
    foo(y_train, y_train_pred)

    print('-'*30)

    print('Model performance for Validation set')
    foo(y_validate, y_validate_pred)

    # r2_list.append(model_test_r2)
    
    print('='*35 + '\n')

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Pipeline(steps=[('Linear Regression', LinearRegression())])
Model performance for Training set
- Root Mean Squared Error: 5.4253
- Mean Absolute Error: 4.3312
- R2 Score: 0.8759
------------------------------
Model performance for Validation set
- Root Mean Squared Error: 5.2016
- Mean Absolute Error: 4.1844
- R2 Score: 0.8781

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Pipeline(steps=[('Lasso', Lasso())])
Model performance for Training set
- Root Mean Squared Error: 6.6537
- Mean Absolute Error: 5.2845
- R2 Score: 0.8133
------------------------------
Model performance for Validation set
- Root Mean Squared Error: 6.1113
- Mean Absolute Error: 4.8095
- R2 Score: 0.8317

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Pipeline(steps=[('Ridge', Ridge())])
Model performance for Training set
- Root Mean Squared Error: 5.4103
- Mean Absolute Error: 4.3271
- R2 Score: 0.8766
------------------------------


In [29]:
for model in model_list:
    y_test_pred = model.predict(X_test)

    print(model)
    print('Model performance for Test dataset')
    foo(y_test, y_test_pred)
    print('='*35 + '\n')

Pipeline(steps=[('Linear Regression', LinearRegression())])
Model performance for Test dataset
- Root Mean Squared Error: 5.2970
- Mean Absolute Error: 4.1681
- R2 Score: 0.8682

Pipeline(steps=[('Lasso', Lasso())])
Model performance for Test dataset
- Root Mean Squared Error: 6.2898
- Mean Absolute Error: 4.8899
- R2 Score: 0.8142

Pipeline(steps=[('Ridge', Ridge())])
Model performance for Test dataset
- Root Mean Squared Error: 5.2771
- Mean Absolute Error: 4.1426
- R2 Score: 0.8692

Pipeline(steps=[('K-Neighbors Regressor', KNeighborsRegressor())])
Model performance for Test dataset
- Root Mean Squared Error: 6.4007
- Mean Absolute Error: 5.1440
- R2 Score: 0.8076

Pipeline(steps=[('Decision Tree', DecisionTreeRegressor())])
Model performance for Test dataset
- Root Mean Squared Error: 9.0540
- Mean Absolute Error: 7.0850
- R2 Score: 0.6150

Pipeline(steps=[('Random Forest Regressor', RandomForestRegressor())])
Model performance for Test dataset
- Root Mean Squared Error: 6.0349
- M